In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [7]:
def normalize(data):
    mean = data.mean()
    std = data.std()
    return (data - mean) / std

In [8]:
def data(split=0.8):
    x = pd.read_csv('train.csv', header=None)
    x = normalize(x)
    x = x.fillna(x.mean())
    y = pd.read_csv('train_labels.csv', header=None)
    x['y'] = y
    x = x.sample(frac=1)
    index = int(split * x.shape[0])
    x_train = x[:index]
    x_test = x[index:]
    y_train = x_train['y']
    y_test = x_test['y']
    del x_train['y']
    del x_test['y']
    return x_train.as_matrix(), y_train.as_matrix(), x_test.as_matrix(), y_test.as_matrix()

In [9]:
def train(x, y, x_test, y_test):
    feature_columns = [tf.feature_column.numeric_column("x", shape=[100])]
    classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units=[10, 20, 10],
                                            n_classes=2,
                                            model_dir="./model")
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": x},
        y=y,
        num_epochs=None,
        shuffle=True)
    classifier.train(input_fn=train_input_fn, steps=2000)
    
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": x_test},
        y=y_test,
        num_epochs=1,
        shuffle=False)
    accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
    print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

In [10]:
x_train, y_train, x_test, y_test = data()
x_train
train(x_train, y_train, x_test, y_test)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_session_config': None, '_log_step_count_steps': 100, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_model_dir': './model', '_keep_checkpoint_max': 5, '_tf_random_seed': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./model/model.ckpt-2000
INFO:tensorflow:Saving checkpoints for 2001 into ./model/model.ckpt.
INFO:tensorflow:loss = 21.2386, step = 2001
INFO:tensorflow:global_step/sec: 560.539
INFO:tensorflow:loss = 3.4978, step = 2101 (0.179 sec)
INFO:tensorflow:global_step/sec: 565.764
INFO:tensorflow:loss = 0.0585824, step = 2201 (0.177 sec)
INFO:tensorflow:global_step/sec: 523.659
INFO:tensorflow:loss = 0.458516, step = 2301 (0.191 sec)
INFO:tensorflow:global_step/sec: 543.466
INFO:tensorflow:loss = 0.0499312, step = 2401 (0.184 sec)
INFO:tensorflow:global_step/sec: 561.574
INFO:tensorfl